## Install Azure Form Recognizer Client Library
```
pip3 install --upgrade azure-ai-formrecognizer
```

## Start an analyze request for your local files

In [4]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import FormRecognizerClient, FormField

FORMRECOGNIZER_ENDPOINT = 'YOUR_ENDPOINT_FROM_CREATED_RESOURCE'
FORMRECOGNIZER_KEY = 'YOUR_KEY_FROM_CREATED_RESOURCE'

# Initiate client with given endpoint and credential
client = FormRecognizerClient(FORMRECOGNIZER_ENDPOINT, AzureKeyCredential(FORMRECOGNIZER_KEY))

# Read the sample image file into memory
IMAGE_FILE = 'simple-invoice.png'
with open(IMAGE_FILE, 'rb') as f:
    data = f.read()

# Send request to Form Recognizer service to process data
task = client.begin_recognize_invoices(data)

# Get the analyze result
analyze_result = task.result()
print('Status: {}, Documents: {}'.format(task.status(), len(analyze_result)))

Status: succeeded, Documents: 1


## You can also analyze files on internet with urls

In [5]:
IMAGE_URL = 'https://raw.githubusercontent.com/Azure-Samples/cognitive-services-REST-api-samples/master/curl/form-recognizer/simple-invoice.png'

# Send request to Form Recognizer service to process data
task = client.begin_recognize_invoices_from_url(IMAGE_URL)

# Get the analyze result
analyze_result = task.result()
print('Status: {}, Documents: {}'.format(task.status(), len(analyze_result)))

Status: succeeded, Documents: 1


## Extract information from analyze result

In [3]:
def show_field(field):
    if field.value_type == 'list':
        for sub_field in field.value:
            show_field(sub_field)
        return
    
    if field.value_type == 'dictionary':
        for sub_field in field.value.values():
            show_field(sub_field)
        return

    if field.value_data is not None:
        print('{:1.2f} | {:<30} | {:<}'.format(field.confidence, field.name, field.value_data.text))
    else:
        print('{:1.2f} | {:<30} | {:<}'.format(field.confidence, field.name, field.value))

for idx, result in enumerate(analyze_result):
    print('Page: {}'.format(idx))
    for field in result.fields.values():
        show_field(field)

Page: 0
0.954 | CustomerAddress                | 1020 Enterprise Way Sunnayvale, CA 87659
0.981 | CustomerAddressRecipient       | Microsoft
0.981 | CustomerName                   | Microsoft
0.981 | DueDate                        | 6/24/2017
0.963 | InvoiceDate                    | 6/18/2017
0.962 | InvoiceId                      | 34278587
0.848 | Amount                         | $56,651.49
0.407 | Date                           | 6/18/2017
0.598 | ProductCode                    | 34278587
0.696 | Tax                            | PT
0.946 | VendorAddress                  | 1 Redmond way Suite 6000 Redmond, WA 99243
0.981 | VendorName                     | Contoso
